In [1]:
!pip install requests-html lxml_html_clean

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 820.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.2/144.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.2.3
    Uninstalling urllib3-2.2.3:
      Successfully uninstalled urllib3-2.2.3


In [314]:
from bs4 import BeautifulSoup
from requests_html import AsyncHTMLSession
import pandas as pd
import io
from time import sleep

In [315]:
session = AsyncHTMLSession()
base_url = "https://www.scimagojr.com/journalrank.php"

In [316]:
params = {
    "type" : "j",
    "openaccess" : "false",
    "area" : 3500
}

In [317]:
r = await session.get(base_url, params=params)
r.status_code

200

In [318]:
download_button_link = list(r.html.find("div[class = ranking_body] a[class = button][href *= out]", first=True).absolute_links)[0]
downloaded_data = await session.get(download_button_link)
downloaded_data.status_code

200

In [319]:
df = pd.read_csv(io.StringIO(downloaded_data.text), delimiter=";", index_col="Rank")
df["Journal's link"] = df["Sourceid"].apply(lambda id: f"https://www.scimagojr.com/journalsearch.php?q={id}&tip=sid&clean=0")

In [320]:
pd.set_option('display.max_columns', None)
df.head()

,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2023),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,%Female,Overton,SDG,Country,Region,Publisher,Coverage,Categories,Areas,Journal's link
Rank,,,,,,,,,,,,,,,,,,,,,,,,
1,22913,Periodontology 2000,journal,"09066713, 16000757","4,744",Q1,151,84,149,11617,3214,146,"18,31","138,30","29,29",0,6,Denmark,Western Europe,Blackwell Munksgaard,1993-2023,Periodontics (Q1),Dentistry,https://www.scimagojr.com/journalsearch.php?q=...
2,19700180533,International journal of oral science,journal,"20493169, 16742818","2,596",Q1,64,56,135,5039,1784,135,"10,47","89,98","44,01",0,14,China,Asiatic Region,Sichuan University Press,2009-2023,Dentistry (miscellaneous) (Q1),Dentistry,https://www.scimagojr.com/journalsearch.php?q=...
3,25669,Journal of Clinical Periodontology,journal,"1600051X, 03036979","2,249",Q1,177,166,447,7866,3347,431,"6,52","47,39","40,06",0,29,Denmark,Western Europe,Blackwell Munksgaard,1974-2023,Periodontics (Q1),Dentistry,https://www.scimagojr.com/journalsearch.php?q=...
4,25610,International Endodontic Journal,journal,"13652591, 01432885","2,155",Q1,147,153,524,8603,2866,457,"5,60","56,23","47,44",0,8,United Kingdom,Western Europe,Wiley-Blackwell Publishing Ltd,"1967-1972, 1974-2023",Dentistry (miscellaneous) (Q1),Dentistry,https://www.scimagojr.com/journalsearch.php?q=...
5,25684,Journal of Dental Research,journal,00220345,"1,909",Q1,211,158,564,6042,4192,518,"6,44","38,24","43,50",2,38,United States,Northern America,SAGE Publications Inc.,"1919-1924, 1927-1935, 1937-2023",Dentistry (miscellaneous) (Q1),Dentistry,https://www.scimagojr.com/journalsearch.php?q=...


In [321]:
df.shape

(235, 24)

In [323]:
information_data = []
scope = []

i = 0
for link in df["Journal's link"].tolist():
  i += 1
  print(f"getting data from link #{i} : {link}")
  j_i = await session.get(link)
  if j_i.status_code != 200:
    print(f"error getting data from link #{i} : {link}")
    information_data.append({})
    scope.append("")
    continue

  journal_information = j_i.html.find("a[id = question_journal]")
  information_data.append({x.text if "mailto" not in x.attrs["href"] else "Email" :x.attrs["href"]  for x in journal_information})
  try:
    scope.append(j_i.html.find("div[class = fullwidth] h2", first=True).html.split("\n", 1)[1].strip().replace("&#13;", ""))
  except:
    scope.append("")

  if i % 10 == 0:
    sleep(2)

getting data from link #1 : https://www.scimagojr.com/journalsearch.php?q=22913&tip=sid&clean=0
getting data from link #2 : https://www.scimagojr.com/journalsearch.php?q=19700180533&tip=sid&clean=0
getting data from link #3 : https://www.scimagojr.com/journalsearch.php?q=25669&tip=sid&clean=0
getting data from link #4 : https://www.scimagojr.com/journalsearch.php?q=25610&tip=sid&clean=0
getting data from link #5 : https://www.scimagojr.com/journalsearch.php?q=25684&tip=sid&clean=0
getting data from link #6 : https://www.scimagojr.com/journalsearch.php?q=24408&tip=sid&clean=0
getting data from link #7 : https://www.scimagojr.com/journalsearch.php?q=26177&tip=sid&clean=0
getting data from link #8 : https://www.scimagojr.com/journalsearch.php?q=24291&tip=sid&clean=0
getting data from link #9 : https://www.scimagojr.com/journalsearch.php?q=91796&tip=sid&clean=0
getting data from link #10 : https://www.scimagojr.com/journalsearch.php?q=26173&tip=sid&clean=0
getting data from link #11 : http

In [324]:
df_res = df.copy()

In [328]:
information_df = pd.DataFrame(information_data)
df_res["Homepage"] = information_df["Homepage"]
df_res["How to Publish"] = information_df["How to publish in this journal"]
df_res["Email"] = information_df["Email"]
df_res["Scope"] = scope

In [329]:
df_res.to_excel("Dentistry_journals.xlsx")